## Area-perimeter self-propelled Voronoi model

After the toy example of notebook 02, let's try to implement a slightly more complicated model, the self-propelled Voronoi area-perimeter Voronoi (VAP) model of [Bi et al., 2016](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.6.021011). This 2D model comprises most of the ingredients we will see in more general simulations, from a coding perspective.

In brief, in the VAP, cells are modeled as the Voronoi tesselation for a series of centroids $\mathbf{v}_i$ (our triangulation vertices). Their overdamped dynamics comprises two terms: self-propulsion and relaxation of an elastic energy:
$$\partial_t \mathbf{v}_i = -\nabla_{\mathbf{v}_i} E_{AP} + v_0 \hat{\mathbf{n}}_i$$
For each cell $i$, $\hat{\mathbf{n}}_i$ is a unit vector (so we will represent it by an angle $\theta_i$) that determines the direction of motion. Units of time are chosen so that the coefficient of $\nabla E_{AP}$ is $1$.
The energy is defined in terms of the Voronoi area $a_i$ and Voronoi perimeter $p_i$ of each cell:
$$E_{AP} = \sum_i k_a(a_i-a_0)^2 + k_p(p_i-p_0)^2 $$
where $k_a, k_p$ are elastic constants, and $a_0, p_0$ are the target area and perimeter. They define the "shape index" $s_0= p_0/\sqrt{a_0}$. The key physics is that above a critical shape index $s_0^*$, the model has a degenerate set of ground states, since for a large $p_0$, there are many polygons with the given target area and perimeter (think floppy balloon).

The orientation $\theta_i$ of each cell is also dynamic. It undergoes rotational diffusion:
$$d\theta_i = D_\theta dW_{t, i} $$
where $dW_{t,i}$ is Brownian motion, independent for each cell $i$, and $D_\theta$ is the diffusion constant.

#### Numerics

The cell array connectivity will be represented by a `HeMesh` (see notebook 01). The geometry is fully described by the triangulation vertex positions, the Voronoi cell centroids. We also need a scalar vertex attribute for the angle $\theta_i$.

To numerically calculate the energy $E_{AP}$, we can obtain Voronoi area and perimeter for each mesh "corner" using the  `triangulax.trigonometry` module. Then we can use the gather/scatter operation `triangulax.meshsum_he_to_vertex_opposite` to sum all corners belonging to a cell (see notebook 01, "Computing cell areas, perimeters, etc via corners"). 
Boundary cells can be handled by "mirroring", i.e., all corners count twice when computing the area/perimeter. Given the energy, JAX autodiff gives us the gradients.

To time-evolve the mesh geometry, we can use `diffrax`, like in notebook 02. `diffrax` can also deal with SDEs, like the Langevin equation for cell angles.
After each timestep, we need to check if the Voronoi edge lengths are below some threshold (the edge lengths can be computed on the fly), and, if so, we need to carry out edge flips. See notebook 01. 
We need to ensure that we do not immidiately "re-flip" an edge. This could be done, for example, via "cool down" period (an edge flipped at step $t$ cannot be flipped again for the next few steps), or by calculating if the edge is shrinking or growing.

It would also be great to generate some visualizations of the time evolution of the mesh using the `cellplot` function, maybe with a user-controlled slider to show the different time steps.

The code should respect the coding style (JAX-compatibility, type hints, etc) used in previous notebooks. To start, let's define the energy and check that relaxation of the energy leads to a state where the $a_i=a_0$ and $p_i=p_0$ constraints are fullfilled (as good as possible).

